# Phase 1

### 1. Identification of the data problem or objective and dataset selection
All the background information for the project will be in a PDF file in the resources folder.

### 2. Data extraction or collection


#### 2.1 Read the data from different sources (CSV files)

In [1]:
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine, text
import pandas as pd

# Load environment variables
load_dotenv()

db_staging_user = os.getenv("DB_STAGING_USER")
db_staging_password = os.getenv("DB_STAGING_PASSWORD")
db_staging_host = os.getenv("DB_STAGING_HOST")
db_staging_port = int(os.getenv("DB_STAGING_PORT", 5432))
db_staging_name = os.getenv("DB_STAGING_NAME")

resources_path = os.getenv("RESOURCES_PATH")
mascotas_propietarios_filename = "Mascotas_Propietarios_despensaAnimal_Generated.csv"
propietarios_transacciones_filename = "Propietarios_Transacciones_despensaAnimal_Generated.csv"

if db_staging_user is None:
    raise ValueError("DB_STAGING_USER is not set")
if db_staging_password is None:
    raise ValueError("DB_STAGING_PASSWORD is not set")
if db_staging_host is None:
    raise ValueError("DB_STAGING_HOST is not set")
if db_staging_port is None:
    raise ValueError("DB_STAGING_PORT is not set")
if db_staging_name is None:
    raise ValueError("DB_STAGING_NAME is not set")
if resources_path is None:
    raise ValueError("RESOURCES_PATH is not set")

connStaging = psycopg2.connect(
    dbname=db_staging_name,
    user=db_staging_user,
    password=db_staging_password,
    host=db_staging_host,
    port=db_staging_port
)
connStaging.autocommit = True   


#### 2.2 Create the database engine for the staging data

In [2]:
db_staging_engine = create_engine(f"postgresql://{db_staging_user}:{db_staging_password}@{db_staging_host}:{db_staging_port}/{db_staging_name}")

#### 2.3 Read the file "Mascotas_Propietarios_despensaAnimal_Generated.csv" and store it in a database table

In [3]:
df_mascotas_propietarios = pd.read_csv(f"{resources_path}/{mascotas_propietarios_filename}", delimiter=",", skiprows=1, low_memory=False)

# Rename columns after loading the data
df_mascotas_propietarios.columns = [
    "nombre_mascota",
    "raza",
    "peso",
    "fecha_nacimiento",
    "sexo",
    "temperamento",
    "numero_carnet",
    "estado_reproductivo",
    "numero_partos",
    "color",
    "fecha_fallecimiento",
    "motivo_fallecimiento",
    "comentarios_fallecimiento",
    "nombre_propietario",
    "ciudad",
    "direccion",
    "telefono",
    "whatsapp",
    "email",
    "tipo_documento",
    "numero_documento",
    "profesion",
    "estado",
    "notificaciones_whatsapp"
    ]

# Just print the first 20 rows to test
df_mascotas_propietarios.head(20)

# Store the CSV file into a database 
with db_staging_engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS mascotas_propietarios_staging (
            nombre_mascota CHARACTER VARYING,
            raza CHARACTER VARYING,
            peso CHARACTER VARYING,
            fecha_nacimiento DATE,
            sexo CHARACTER VARYING,
            temperamento CHARACTER VARYING,
            numero_carnet CHARACTER VARYING,
            estado_reproductivo CHARACTER VARYING,
            numero_partos CHARACTER VARYING,
            color CHARACTER VARYING,
            fecha_fallecimiento DATE,
            motivo_fallecimiento CHARACTER VARYING,
            comentarios_fallecimiento CHARACTER VARYING,
            nombre_propietario CHARACTER VARYING,
            ciudad CHARACTER VARYING,
            direccion CHARACTER VARYING,
            telefono CHARACTER VARYING,
            whatsapp CHARACTER VARYING,
            email CHARACTER VARYING,
            tipo_documento CHARACTER VARYING,
            numero_documento CHARACTER VARYING,
            profesion CHARACTER VARYING,
            estado CHARACTER VARYING,
            notificaciones_whatsapp CHARACTER VARYING
        );
    """))
    conn.commit()
    print("Table 'mascotas_propietarios_staging' created successfully.")

# Store DataFrame into PostgreSQL table named 'mascotas_propietarios_staging'
df_mascotas_propietarios.to_sql("mascotas_propietarios_staging", db_staging_engine, if_exists="append", index=False)

print("The CSV file data was inserted successfully!")

Table 'mascotas_propietarios_staging' created successfully.
The CSV file data was inserted successfully!


#### 2.4 Get data from the "mascotas_propietarios_staging" table to test

In [4]:
# Just print the first 20 rows to test
df_mascotas_propietarios_verification = pd.read_sql("SELECT * FROM mascotas_propietarios_staging LIMIT 20;", db_staging_engine)
df_mascotas_propietarios_verification

,nombre_mascota,raza,peso,fecha_nacimiento,sexo,temperamento,numero_carnet,estado_reproductivo,numero_partos,color,...,ciudad,direccion,telefono,whatsapp,email,tipo_documento,numero_documento,profesion,estado,notificaciones_whatsapp
0,THANOS URREGO,Poodle,None,2022-09-01,Macho,social,1631,Entero,None,None,...,Cali,CRA100 #28-68 VALLE DEL LILI MADEIRO TORRE 1 ...,None,3207201137,lizethurrego1990@gmail.com,CC,1036637677,None,Activo,Activo
1,TINENK ORDONEZ,Shit-zu,None,2017-02-17,Hembra,social,1650,None,None,None,...,Santiago de Cali,cra 85 c # 33- 40 casa 54,3058147632,3157929392,pa_ordonez@hotmail.com,CC,66987417,None,Activo,Activo
2,SIMON VACCA,PUG,None,2021-08-01,Macho,social,1184,None,None,None,...,None,Cra98B #45-200 SAN MIGUEL,None,3234191060,None,CC,None,None,Activo,Activo
3,SUSSY PUERTA,YORK SHIRE TERRIER,3.6,2016-10-07,Hembra,social,1359,ENTERA,NINGUNO,None,...,None,Cra. 98b #34-53 GUADALQUIVIR CASA 64,3134024437,3146096191,None,CC,1006107262,None,Activo,Activo
4,EEVEE ANACONA,Poodle,None,2017-09-03,Hembra,social,1632,None,None,None,...,Cali,"Calle 45#98B-65 Apto 403, torre 8",None,3216066041,jazmin.dag@gmail.com,CC,1061750508,None,Activo,Activo
5,DULCE RODRIGUEZ,PINSCHER,None,2020-04-01,Macho,social,1628,Esterilizado,None,None,...,Cali,CRA98B#48-127 VALLE DEL LILI,None,3175175266,leidyjoroma@gmail.com,CC,2569816,None,Activo,Activo
6,EURO LONDOÑO,SPHYNX,None,None,Macho,social,1634,None,None,None,...,CALI,Calle 13 #98 -10 Multicentro 2 Apto 801 torre 1,None,3225529091,None,CC,None,None,Activo,Activo
7,MOLLY GIRALDO,Mestizo,None,2018-10-15,Hembra,social,1636,None,None,None,...,None,CLL 28 # 96-186 APTO 301 TORRE D PORTAL DEL LI...,None,3164403237,None,CC,1116275102,None,Activo,Activo
8,MAXIMILIANO BOHORQUEZ,ANGORA,None,None,Macho,social,1657,None,None,None,...,None,Cra 98b # 34-53 Guadalquivir casa 76,None,3186156608,Memis0914@gmail.com,CC,1.023.865.148,None,Activo,Activo
9,LUPE GIRALDO,Poodle,None,2018-04-14,Hembra,social,1638,None,None,None,...,None,CLL 28 # 96-186 APTO 301 TORRE D PORTAL DEL LI...,None,3164403237,None,CC,1116275102,None,Activo,Activo


#### 2.5 Read the file "Propietarios_Transacciones_despensaAnimal_Generated.csv" and store it in a database table

In [5]:
df_propietarios_transacciones = pd.read_csv(f"{resources_path}/{propietarios_transacciones_filename}", delimiter=",", skiprows=1, dtype={"numero_documento": str}, low_memory=False)

# Rename columns after loading the data
df_propietarios_transacciones.columns = [
    "nombre_propietario",
    "tipo_documento",
    "numero_documento",
    "nombre_mascota",
    "servico_prestado",
    "valor_servicio",
    "fecha_servicio"
    ]

# Just print the first 20 rows to test
df_propietarios_transacciones.head(20)

# Store the CSV file into a database 
with db_staging_engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS propietarios_transacciones_staging (
            nombre_propietario CHARACTER VARYING,
            tipo_documento CHARACTER VARYING,
            numero_documento CHARACTER VARYING,
            nombre_mascota CHARACTER VARYING,
            servico_prestado CHARACTER VARYING,
            valor_servicio NUMERIC,
            fecha_servicio DATE            
        );
    """))
    conn.commit()
    print("Table 'propietarios_transacciones_staging' created successfully.")

# Store DataFrame into PostgreSQL table named 'propietarios_transacciones_staging'
df_propietarios_transacciones.to_sql("propietarios_transacciones_staging", db_staging_engine, if_exists="append", index=False)

print("The CSV file data was inserted successfully!")

Table 'propietarios_transacciones_staging' created successfully.
The CSV file data was inserted successfully!


#### 2.6 Get data from the "propietarios_transacciones_staging" table to test

In [6]:
# Just print the first 20 rows to test
df_propietarios_transacciones = pd.read_sql("SELECT * FROM propietarios_transacciones_staging LIMIT 20;", db_staging_engine)
df_propietarios_transacciones

,nombre_propietario,tipo_documento,numero_documento,nombre_mascota,servico_prestado,valor_servicio,fecha_servicio
0,NICOLAS LOPEZ CANO,CC,1116733904,TOMAS LOPEZ CANO,Otros,814686.0,2017-11-22
1,NICOLAS LOPEZ CANO,CC,1116733904,TOMAS LOPEZ CANO,Baño,936227.0,2019-10-03
2,NICOLAS LOPEZ CANO,CC,1116733904,TOMAS LOPEZ CANO,Venta Alimentos,998379.0,2019-05-15
3,NICOLAS LOPEZ CANO,CC,1116733904,TOMAS LOPEZ CANO,Vacunación,1207885.0,2023-03-11
4,NICOLAS LOPEZ CANO,CC,1116733904,TOMAS LOPEZ CANO,Peluquería,464505.0,2022-09-30
5,NICOLAS LOPEZ CANO,CC,1116733904,TOMAS LOPEZ CANO,Desparacitada,375229.0,2024-05-08
6,NICOLAS LOPEZ CANO,CC,1116733904,TOMAS LOPEZ CANO,Desparacitada,800173.0,2018-05-09
7,NICOLAS LOPEZ CANO,CC,1116733904,TOMAS LOPEZ CANO,Baño,437541.0,2020-11-06
8,NICOLAS LOPEZ CANO,CC,1116733904,TOMAS LOPEZ CANO,Vacunación,1150973.0,2017-12-05
9,NICOLAS LOPEZ CANO,CC,1116733904,TOMAS LOPEZ CANO,Peluquería,1612722.0,2019-05-02
